# **Mail automation requirement**


Description : this utility is providing automation of any Mail sent to the user. We will have a list of Mails and Subjects present in the sheet. We need to build a python based utility which will take input of sheet data and it will automatically send mail to that list of mail mentioned in the sheet.

Email Id and Subjects need to be provided in the sheet. The rest of the information is fixed like the Mail message body and attachment is the same for all the mails.

At the end we will have a response sheet where we need to mention mail id with status of Mail delivery (eg Sent successfully, Failed, Pending etc.)

In [194]:
# importing necessary libraries
import smtplib  # Library for sending emails via SMTP
import pandas as pd  # Library for handling spreadsheet data (reading/writing Excel/CSV files)
import email  # Library for managing email messages
import openpyxl  # Library for handling Excel (.xlsx) files

In [196]:
# SMTP server configuration for Gmail
SMTP_SERVER = "smtp.gmail.com"  # Gmail SMTP server
SMTP_PORT = 587  # Port for TLS encryption
SENDER_EMAIL = "hemabutani75@gmail.com"  # Sender's email address
SENDER_PASSWORD = "vqjd hkoh zapu pdjn"  # App password for authentication
ATTACHMENT_PATH = r"C:\Users\RajakHem\Desktop\Power BI project\My Python Project\Hema_Butani.pdf"  # File to attach

In [181]:
df = pd.read_excel(r"C:\Users\RajakHem\Desktop\Power BI project\My Python Project\email_list.xlsx") #uploading excel file having all mail and subject list

In [198]:
from email.mime.multipart import MIMEMultipart  # Used for creating email structure
from email.mime.text import MIMEText  # Used to add text content to the email
from email.mime.base import MIMEBase  # Used for handling file attachments
from email import encoders  # Used to encode file attachments     

In [200]:
def send_email(receiver_email, subject):
    try:
        # # Creating the email message
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL  # Sender's email address
        msg['To'] = receiver_email  # Receiver's email address
        msg['Subject'] = subject     # Email subject

        # Email Body (Fixed for all)
        body = "I am writing to express my interest in the Data Analyst position. With a background in data analysis, proficiency in statistical techniques, and a passion for deriving actionable insights from complex datasets, I am confident in my ability to contribute effectively to your team."
        msg.attach(MIMEText(body, 'plain'))

        # Adding attachment
        try:
            attachment = open(ATTACHMENT_PATH, "rb")
            part = MIMEBase('application', 'octet-stream')  # Creating a binary attachment
            part.set_payload(attachment.read()) # Reading the file content
            encoders.encode_base64(part)  # Encoding the attachment in base64
            part.add_header('Content-Disposition', f"attachment; filename={ATTACHMENT_PATH.split('/')[-1]}")
            msg.attach(part)
            attachment.close()
        except Exception as e:
            print(f"Attachment Error: {e}")

    # Establish connection to SMTP server and send email
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()  # Secure the connection
        server.login(SENDER_EMAIL, SENDER_PASSWORD) # Login to the email account
        server.sendmail(SENDER_EMAIL, receiver_email, msg.as_string())  # Sending the email
        server.quit() # Closing the connection

        return "Sent successfully"
    except Exception as e:
        return f"Failed: {str(e)}"

In [202]:
status_list = [] # List to store email statuses

for index, row in df.iterrows():
    email = row['Email ID'] # Extract recipient's email address from the row
    subject = row['Subject']  # Extract email subject from the row
    
    status = send_email(email, subject) # Call the send_email function and store the result
    status_list.append([email, status]) # Append email and status to the list

# Convert the list into a DataFrame with two columns: 'Email ID' and 'Status'
df_status = pd.DataFrame(status_list, columns=['Email ID', 'Status'])

# Save the DataFrame as an Excel file to record the email sending status
df_status.to_excel("email_status.xlsx", index=False)

# Print message after completion
print("Email sending completed. Check email_status.xlsx for results.")


Email sending completed. Check email_status.xlsx for results.
